In [9]:
# read in the puzzle data from the pickle file
import pickle
import torch
import torch.nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Discriminator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # make a discriminator now with input being 1x72
        self.dense = torch.nn.Linear(72, 1)
        self.conv1 = torch.nn.Conv2d(1, 64, (5, 5), padding='same')
        self.BN1 = torch.nn.BatchNorm2d(64)
        self.conv2 = torch.nn.Conv2d(64, 1, (5, 5), padding='same')
        self.BN2 = torch.nn.BatchNorm2d(1)

        self.relu = torch.nn.ReLU()
        self.sig = torch.nn.Sigmoid()

    def forward(self, x):
        batchSize = x.size()[0]
        pieces = x[..., 0:64]
        extraCrap = x[..., 64:]
        # now reshpae the pieces part into an 8x8 square
        pieces = torch.reshape(pieces, (batchSize, 1, 8, 8))
        # do a conv now
        out = self.conv1(pieces)
        out = self.relu(out)
        out = self.BN1(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.BN2(out)
        out = torch.reshape(out, (batchSize, 64))
        out = torch.cat([out, extraCrap], dim=1)
        return self.sig(self.dense(out))

dataLimit = 10000
with open('./KaggleData/encodedPuzzles.pickle', 'rb') as f:
    puzzles = pickle.load(f)

puzzles = puzzles[0:dataLimit]
N = len(puzzles)
print("Training on " + str(N) + " Board Positions")
# convert board pos to a tensor now
boardTensors = torch.stack([torch.Tensor(puzzles[i]) for i in range(N)])
trainBoardTensors = boardTensors[0:N]
# zscore it
mean = trainBoardTensors.mean(dim=0)
std = trainBoardTensors.std(dim=0)
trainBoardTensors = (trainBoardTensors - mean) / std

Training on 10000 Board Positions


In [13]:
import numpy as np
import matplotlib.pyplot as plt

# make an roc curve on the real data
# load the discriminator
disc = torch.load('disc.pth')
# run the discriminator on the real data
for i in range(N):
    print("Running on " + str(i))
    # run the discriminator on the real data
    realData = trainBoardTensors[i]
    realData = torch.reshape(realData, (1, 72))
    realData = disc(realData)
    if i == 0:
        realDataTensor = realData
    else:
        realDataTensor = torch.cat((realDataTensor, realData), dim=0)

# now make the roc curve


Running on 0


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor